# Create JSON for HRRR Grib files

In [ ]:
import pandas as pd

In [ ]:
dates = pd.date_range(start='2019-01-01 23:00',end='2019-01-02 00:00', freq='1h')

In [ ]:
%run grib2.py

%run hdf.py

In [ ]:
def date2url(date):
    yyyymmdd = date.strftime('%Y%m%d')
    hh = date.strftime('%H')
    cfile = f's3://noaa-hrrr-bdp-pds/hrrr.{yyyymmdd}/conus/hrrr.t{hh}z.wrfsfcf01.grib2'
    return cfile

In [ ]:
files = [date2url(date) for date in dates]

In [ ]:
files

In [ ]:
os.path.dirname(files[0]).split('/')[-2]

In [ ]:
os.path.dirname(files[0]).split('/')[-2]+os.path.basename(files[0])[-21:]

In [ ]:
%time example_multi(files)

In [ ]:
%%time
fslocal = fsspec.filesystem('')
json_files = fslocal.glob('hrrr.*.t*.json')

In [ ]:
json_files

In [ ]:
mzz = MultiZarrToZarr(json_files, remote_protocol="s3",
                      remote_options={"anon": True}, with_mf='time')
mzz.translate("hrrr.total.json")

Try out the resulting JSON:

In [ ]:
import xarray as xr
import hvplot.xarray

In [ ]:
%%time
so = {"anon": True}
fo = "hrrr_total.json"
fs = fsspec.filesystem("reference", fo=fo, target_options=so,
                       remote_protocol="s3", remote_options=so)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))

In [ ]:
ds.t2m.hvplot.quadmesh(x='longitude', y='latitude', 
                       rasterize=True, geo=True, cmap='turbo', tiles='OSM')